<a href="https://colab.research.google.com/github/Sri2590/Vibot-Chatbot/blob/main/agent_emulator_hybrid_search_rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#nihaal commented here
#Srinidhi commented here
from langchain_openai import ChatOpenAI
from openai import OpenAI, DefaultHttpxClient
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.documents.base import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import chromadb
import sqlite3
from langserve import RemoteRunnable

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
llm = ChatOpenAI(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key="",
    base_url="https://llm.monsterapi.ai/v1/",
    http_client=DefaultHttpxClient(verify = False)
)

In [ ]:
llm.invoke("hi")

In [ ]:
decision_prompt =  PromptTemplate.from_template("""You have access to the following tools:
            1. retrieve -  call this tool when you want to know information to answer a question.
            2. greet - call this tool when the user greets you or asks questions like who are you. In summary when they are having a casual conversation.
            User has asked the following question {query}
            Which tool will you use? just return the name of the tool alone and nothing else.""")

In [ ]:
decision_chain = decision_prompt|llm|StrOutputParser()

In [ ]:
decision_chain.invoke({"query":"Explain relative grading in detail"})

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en")

In [ ]:
persistent_client = chromadb.PersistentClient(path="Vectorstore1")

vectorstore = Chroma(client=persistent_client,collection_name="Questions",embedding_function=embedding_function)

retriever = vectorstore.as_retriever(search_kwargs={'k': 12})
retriever.invoke("What is FC?")

In [ ]:
conn = sqlite3.connect('chunk_database_mixtral.db',check_same_thread=False)
cursor = conn.cursor()

In [ ]:
def retrieve_chunk(chunk_id):
    cursor.execute("SELECT chunk FROM CHUNKS WHERE chunk_id=?", (chunk_id,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
loader = PyPDFLoader("Admission-rewritten-full.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [ ]:
import re
from collections import Counter
from typing import List, Tuple

In [ ]:
class SimpleTextSearch:
    def __init__(self,docs):
        self.__docs =  [doc.page_content for doc in docs]
    def search(self,query,top_k=2):
        query_terms = set(query.lower().split())
        scored_sentences = []
        for sentence in self.__docs:
            # Count matching terms
            sentence_terms = set(sentence.lower().split())
            matching_terms = query_terms.intersection(sentence_terms)

            # Calculate score based on proportion of query terms found
            if matching_terms:
                score = len(matching_terms) / len(query_terms)
                scored_sentences.append((sentence, score))

        # Sort by score and return top k
        scored_sentences.sort(key=lambda x: x[1], reverse=True)
        return scored_sentences[:top_k]

In [ ]:
TextRetriever = SimpleTextSearch(docs=texts)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [ ]:
def retrieve(state: dict) -> str:
    query = state["query"]
    questions = retriever.invoke(query)
    chunk_ids = set()
    for question in questions:
        chunk_ids.add(question.metadata['chunk_id'])
    chunks = []
    for chunk_id in chunk_ids:
        chunks.append(Document(page_content=retrieve_chunk(chunk_id)))
    chunk_string = "\n\n".join([d.page_content for d in chunks])
    keyword_docs = TextRetriever.search(query,2)
    keyword_string = "\n\n".join([d[0] for d in keyword_docs])
    return  f"""Documents retrieved based on semantic similarity:{chunk_string} \n\nDocuments retrieved based on Keyword search:{keyword_string}\n\n"""

In [ ]:
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)

In [ ]:
def retrieve_and_rerank(state: dict) -> str:
    query = state["query"]
    questions = retriever.invoke(query)
    chunk_ids = set()
    for question in questions:
        chunk_ids.add(question.metadata['chunk_id'])
    chunks = []
    for chunk_id in chunk_ids:
        chunks.append(Document(page_content=retrieve_chunk(chunk_id)))
    keyword_docs = TextRetriever.search(query,2)
    for chunk in keyword_docs:
        chunks.append(Document(page_content=chunk[0]))
    reranked_docs = compressor.compress_documents(chunks,query)
    return "\n\n".join([d.page_content for d in reranked_docs])

In [ ]:
retrieval_prompt = PromptTemplate.from_template("""You are VIBOT an assistant designated to answer questions related to VIT university.
                                                   Answer the following question {query}
                                                   Here are some information that will be useful to answer question: {context}
                                                   Answer precisely unless you are asked to answer in detail.
                                                   Try to avoid providing additional unwanted information to users. Answer to the point.""")

In [ ]:
paraphrase_prompt = PromptTemplate.from_template("""You will be provided with a question and and answer that was created to answer the question.
                                                    The answer may have impurities due to a simple text search remove the irrevelent parts from
                                                    the answer that is not related to the question.Just return the pharaphrased answer and nothing.
                                                    In any case remove only the unwanted parts from the answer.
                                                    Question:{query}
                                                    Answer: {answer}""")

In [ ]:
#answer_with_retrieval = RunnableAssign({"answer":retrieval_prompt | llm | StrOutputParser()}) | paraphrase_prompt | llm | StrOutputParser()
answer_with_retrieval = retrieval_prompt | llm | StrOutputParser()

In [ ]:
generic_prompt = PromptTemplate.from_template("""You are VIBOT an assistant designated to answer questions related to VIT university.
                  Here are some information about you:
                  You are developed in IEDC lab at VIT Chennai.
                  You think your developers are legendary.
                  You will not resopnd to harmful or disrespectful queries.
                  User told : {query} respond to the user respectfully.""")

In [ ]:
answer_without_retrieval = generic_prompt | llm | StrOutputParser()

In [ ]:
answer_without_retrieval.invoke({"query":"hi"})

In [ ]:
branch = RunnableBranch(
    (lambda x: "retrieve" in x["action"].lower(),RunnableAssign({"context":RunnableLambda(retrieve_and_rerank)})| answer_with_retrieval),
    answer_without_retrieval,)

In [ ]:
full_chain = RunnableAssign({"action": decision_chain}) | branch

In [ ]:
out = full_chain.invoke({"query":"What is FFCS?"})

In [ ]:
out

In [ ]:
out1 = full_chain.invoke({"query":"hi"})

In [ ]:
out1

In [ ]:
out1 = full_chain.invoke({"query":"In alpha block projector is not working and teachers are facing difficulties could you help!"})

In [ ]:
out1

In [ ]:
type(out1)

In [ ]:
def llama3_chat():
    print("Hello!!!! I am llama3 and I can help with your document. \nIf you want to stop you can enter STOP at any point!")
    print()
    print("-------------------------------------------------------------------------------------")
    question = input()
    while question != "~":
        out  = full_chain.invoke({"query":question})
        print(out)
        print("\nIs there anything else you would like my help with?")
        print("-------------------------------------------------------------------------------------")
        question = input()

In [ ]:
llama3_chat()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("final.csv")

In [ ]:
df.head()

In [ ]:
import tqdm

In [ ]:
output = {}
for i in tqdm.tqdm(range(len(df))):
    try:
        question = df["question"][i]
        correct_answer    = df["answer"][i]
        generated_answer  = full_chain.invoke({"query":question})
        output[question] = [correct_answer,generated_answer]
    except:
        pass

In [ ]:
import json
with open("sample.json", "w") as outfile:
    json.dump(output, outfile)

In [ ]:
output['who are u']

In [ ]:
import csv

In [ ]:
filename = 'output_1.csv'

# Open the file in write mode
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['question','correct_answer','generated_answer'])

    # Write the data rows
    for question, answers in output.items():
        # Assuming the first item in the list is the correct answer and the second is the generated answer
        correct_answer = answers[0]
        answer = answers[1]
        writer.writerow([question,correct_answer,answer])